### 1. Travelling Salesman Problem (TSP)

In [1]:
import random
import numpy as np

class TSPSolver():
    def __init__(self, cost_matrix, population_size=100, generations=500):
        self.population_size = population_size
        self.generations = generations
        self.cost_matrix = np.array(cost_matrix)
        self.cities = len(cost_matrix)
        
    def create_initial_population(self):
        population = []
        for _ in range(self.population_size):
            chromosome = list(range(1, self.cities))
            random.shuffle(chromosome)
            chromosome = [0] + chromosome
            population.append(chromosome)
        return population
    
    def calculate_fitness(self, chromosome):
        total_distance = 0
        for i in range(self.cities-1):
            total_distance += self.cost_matrix[chromosome[i]][chromosome[i+1]]
        total_distance += self.cost_matrix[chromosome[-1]][chromosome[0]]
        return -total_distance
    
    def crossover(self, parent1, parent2):
        point1, point2 = sorted(random.sample(range(1, self.cities), 2))
        
        def create_child(p1, p2):
            child = [-1] * self.cities
            child[0] = 0
            child[point1:point2] = p1[point1:point2]
            remaining = [x for x in p2 if x not in child[point1:point2] and x != 0]
            for i in range(1, self.cities):
                if child[i] == -1:
                    child[i] = remaining.pop(0)
            return child
        
        return create_child(parent1, parent2), create_child(parent2, parent1)
    
    def mutate(self, chromosome, mutation_rate=0.1):
        if random.random() < mutation_rate:
            i, j = random.sample(range(1, self.cities), 2)
            chromosome[i], chromosome[j] = chromosome[j], chromosome[i]
        return chromosome
    
    def solve(self):
        population = self.create_initial_population()
        best_solution = None
        best_fitness = float('-inf')
        
        for generation in range(self.generations):
            fitness_scores = [self.calculate_fitness(chrom) for chrom in population]
            max_fitness_idx = fitness_scores.index(max(fitness_scores))
            
            if fitness_scores[max_fitness_idx] > best_fitness:
                best_fitness = fitness_scores[max_fitness_idx]
                best_solution = population[max_fitness_idx]
            
            new_population = [best_solution]
            
            while len(new_population) < self.population_size:
                parent1 = max(random.sample(population, 5), 
                            key=lambda x: self.calculate_fitness(x))
                parent2 = max(random.sample(population, 5), 
                            key=lambda x: self.calculate_fitness(x))
                
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                new_population.extend([child1, child2])
            
            population = new_population[:self.population_size]
            
        return best_solution, -best_fitness

def main():
    cost_matrix = [
        [0, 10, 15, 20],
        [10, 0, 35, 25],
        [15, 35, 0, 30],
        [20, 25, 30, 0]
    ]
    solver = TSPSolver(cost_matrix)
    route, distance = solver.solve()
    print("\nTSP Solution:")
    print("Route:", " -> ".join(map(str, route + [0])))
    print("Distance:", distance)

if __name__ == "__main__":
    main()


TSP Solution:
Route: 0 -> 2 -> 3 -> 1 -> 0
Distance: 80


### 2. Optimal Power Flow Problem

In [14]:
import random
import numpy as np

class OPFSolver():
    def __init__(self, num_generators, demand, coefficients, limits, population_size=100, generations=500):
        self.num_generators = num_generators
        self.demand = demand
        self.loss = 0.05 * demand
        self.total_required = demand + self.loss
        
        self.a = [c[0] for c in coefficients]
        self.b = [c[1] for c in coefficients]
        self.c = [c[2] for c in coefficients]
        self.p_min = [l[0] for l in limits]
        self.p_max = [l[1] for l in limits]
        
        self.population_size = population_size
        self.generations = generations

    def create_initial_population(self):
        population = []
        for _ in range(self.population_size):
            chromosome = []
            remaining_power = self.total_required
            
            for i in range(self.num_generators - 1):
                max_possible = min(self.p_max[i], remaining_power - sum(self.p_min[i+1:]))
                min_possible = max(self.p_min[i], remaining_power - sum(self.p_max[i+1:]))
                power = random.uniform(min_possible, max_possible)
                chromosome.append(power)
                remaining_power -= power
            
            chromosome.append(remaining_power)
            population.append(chromosome)
        return population

    def calculate_cost(self, outputs):
        total_cost = 0
        for i in range(self.num_generators):
            p = outputs[i]
            total_cost += self.a[i] + self.b[i]*p + self.c[i]*p*p
        return total_cost

    def calculate_fitness(self, chromosome):
        cost = self.calculate_cost(chromosome)
        
        power_imbalance = abs(sum(chromosome) - self.total_required)
        
        limit_violations = sum(max(0, self.p_min[i] - p) + max(0, p - self.p_max[i]) 
                             for i, p in enumerate(chromosome))
        
        penalty = 1e6 * (power_imbalance + limit_violations)
        return -(cost + penalty)

    def crossover(self, parent1, parent2):
        child1, child2 = [], []
        
        for i in range(self.num_generators):
            w = random.random()
            c1 = w * parent1[i] + (1-w) * parent2[i]
            c2 = w * parent2[i] + (1-w) * parent1[i]
            
            child1.append(max(self.p_min[i], min(self.p_max[i], c1)))
            child2.append(max(self.p_min[i], min(self.p_max[i], c2)))
        
        return child1, child2

    def mutate(self, chromosome, mutation_rate=0.1):
            if random.random() < mutation_rate:
                i = random.randrange(self.num_generators)
                chromosome[i] = random.uniform(self.p_min[i], self.p_max[i])
            return chromosome

    def solve(self):
        population = self.create_initial_population()
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(self.generations):
            fitness_scores = [self.calculate_fitness(chrom) for chrom in population]
            max_fitness_idx = fitness_scores.index(max(fitness_scores))
            
            if fitness_scores[max_fitness_idx] > best_fitness:
                best_fitness = fitness_scores[max_fitness_idx]
                best_solution = population[max_fitness_idx]
            
            new_population = [best_solution]
            
            while len(new_population) < self.population_size:
                parent1 = max(random.sample(population, 5), 
                            key=lambda x: self.calculate_fitness(x))
                parent2 = max(random.sample(population, 5), 
                            key=lambda x: self.calculate_fitness(x))
                
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                new_population.extend([child1, child2])
            
            population = new_population[:self.population_size]
            
        return best_solution, self.calculate_cost(best_solution)

def main():
    num_generators = 2
    demand = 100.0
    coefficients = [
        [10.0, 0.5, 0.02],
        [15.0, 0.4, 0.015]
    ]
    limits = [
        [20.0, 80.0],
        [30.0, 120.0]
    ]
    solver = OPFSolver(num_generators, demand, coefficients, limits)
    outputs, cost = solver.solve()
    print("\nOPF Solution:")
    print("Generator Outputs:", [round(x, 2) for x in outputs])
    print("Cost:", round(cost, 2))

if __name__ == "__main__":
    main()


OPF Solution:
Generator Outputs: [43.6, 61.4]
Cost: 165.93
